In [1]:
!git clone https://github.com/ZhangShiyue/RevitalizeCherokee.git

Cloning into 'RevitalizeCherokee'...
remote: Enumerating objects: 2955, done.
remote: Total 2955 (delta 0), reused 0 (delta 0), pack-reused 2955
Receiving objects: 100% (2955/2955), 185.00 MiB | 38.67 MiB/s, done.
Resolving deltas: 100% (2/2), done.
Updating files: 100% (2975/2975), done.


In [1]:
!pwd

/content


In [2]:
%cd RevitalizeCherokee

/content/RevitalizeCherokee


In [3]:
!pip install pybind11
!pip install fastwer
!sudo apt-get install tesseract-ocr-chr
!pip install pytesseract==0.3.9
!sudo apt install tesseract-ocr
!pip install jiwer evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Reading package lists... Done
Building dependency tree       
Reading state information... Done
tesseract-ocr-chr is already the newest version (1:4.00~git30-7274cfa-1).
0 upgraded, 0 newly installed, 0 to remove and 27 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Reading package lists... Done
Building dependency tree       
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2build2).
0 upgraded, 0 newly installed, 0 to remove and 27 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
!tesseract --list-langs

List of available languages (3):
chr
eng
osd


In [5]:
import cv2
import pytesseract
import fastwer
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import Image, display

In [6]:
from evaluate import load
wer = load("cer")
predictions = ["hello world", "good night moon"]
references = ["hello world", "good night mookkkkkkkknlight"]
wer_score = wer.compute(predictions=predictions, references=references)
print(wer_score)

0.3333333333333333


In [7]:
#for i, img in enumerate(img_list):
#  if ('.png' in img):
#    print(f'Display sample image {i+1} - {img}')
#    display(Image(filename=f'./OCR/original/{img}'))

**Optical Character Recognition**

In [69]:
import chardet
def calculate_tesseract_ocr(folder):
  # Show the sample images that we will work on (Need to create a folder to upload sample images in Colab environment)
  img_list = os.listdir(folder)
  img_list = sorted(img_list)
  wer = load("wer")
  cer = load("cer")
  # Create empty dataframe to store output
  df_output = pd.DataFrame(columns = ['img_filename', 'ocr_output','ref_text'])
  # Loop through sample images


  for img in img_list:
    if '.png' in img:
      im = cv2.imread(f'{folder}{img}')
      img_rgb = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
      output = pytesseract.image_to_string(img_rgb,lang="chr")
    #   output = output.replace("\n","")

      file = open(f'{folder}{img.replace(".png",".tesseract")}', "r+")
      file.seek(0)                        # <- This is the missing piece
      file.truncate()
      file.write(output)
      #print(output)
      with open(f'{folder}{img.replace(".png",".txt")}', 'rb') as fil:
        encoding=chardet.detect(fil.read())["encoding"]
        # print(encoding)
        with open(f'{folder}{img.replace(".png",".txt")}', 'r',encoding=encoding) as f:
            data = f.read()
      dictionary = {'img_filename':img, 'ocr_output':output, 'ref_text':data}
      df_output = df_output.append(dictionary, ignore_index=True)
  for index, row in df_output.iterrows():
        filename = row['img_filename']
        ref =[row['ref_text'].replace("\n","").replace("\x0c","").strip()]
        output = [row['ocr_output'].replace("\n","").replace("\x0c","").strip() ]
        # print(ref)
        # print(output)
        cer_out = cer.compute(predictions=output, references=ref)
        wer_out = wer.compute(predictions=output, references=ref)
        df_output.loc[df_output['img_filename'] == filename, 'cer'] = round(cer_out,2) # Round value to 2 decimal places
        df_output.loc[df_output['img_filename'] == filename, 'wer'] = round(wer_out,2)
  # Overall performances
  mean_cer = df_output['cer'].mean()
  mean_wer = df_output['wer'].mean()
  print(f'Mean CER = {mean_cer}, Mean WER = {mean_wer}')
  med_cer = df_output_original['cer'].median()
  med_wer = df_output_original['wer'].median()
  print(f'Median of all setences CER = {med_cer}, Median of all setences WER = {med_wer}')
  return df_output


In [70]:
df_output_original = calculate_tesseract_ocr("./OCR/original/")

Mean CER = 0.8564999999999999, Mean WER = 1.3555000000000001
Median of all setences CER = 0.49, Median of all setences WER = 0.865


In [71]:
df_output_original

,img_filename,ocr_output,ref_text,cer,wer
0,Charlie_Brown_Christmas_sample1.png,-Ᏺ-\nᏭ\n\nᏃ\nᏝ\n\nᎫ ……\nᎠᎥ\n\nᎢ\n\nᏴᏒ\n\n-Ꮞ\n\...,"“Ꮩ ᎢᏳᏍᏗ ᏣᏚᎵᎭ?” ᎤᏛᏛᏅ ᎷᏏ.\n\n“ᏥᏳᎪᏗ ᎢᎦᏛᏁᏗ,” ᎤᏛᏅ Ꮳ...",0.28,0.42
1,Charlie_Brown_Christmas_sample2.png,ᏎᎩᏊᏃ ᏂᎦᏓ ᏧᎾᏁᎶᏗ ᏗᎪᏪᎵ ᏚᏂᎾᎩᏒ. ᏃᎢ\nᎤᏍᏆᎸᎲ ᏣᎵ ᎤᎧᏗᎨ Ꮷ...,ᏎᎩᏊᏃ ᏂᎦᏓ ᏧᎾᏁᎶᏗ ᏗᎪᏪᎵ ᏚᏂᎾᎩᏒ. ᏃᎢ\nᎤᏍᏆᎸᎲ ᏣᎵ ᎤᏬᏗᎨ Ꮷ...,0.05,0.28
2,ClimbingTheAppleTree_sample1.png,"ᎠᏥᏃᎯᏎᎲ ᎢᏳᏩᏁᏗ ᎤᏣᎴᏍᏓ, ᎤᏩᎨᏫᏎᎢ “ᏃᏗ ᏓᏥᏇᏅᏂ\nᎤᏪᎯᏍᏓᏁᎲ ...","ᎠᏥᏃᎯᏎᎲ ᎢᏳᏩᏁᏗ ᎤᏣᎴᏍᏓ, ᎤᏩᎨᏫᏎᎢ\nᎤᏪᎯᏍᏓᏁᎲ ᎧᏃᎨᏂ.\n\n“...",0.39,0.70
3,ClimbingTheAppleTree_sample2.png,ᎾᎾ\n-Ꮈ\nᎾ\nᎾ\nᎨᎸ\nᏨ\nᏝᏃ\n\n..ᎤᎵᏍᏆᎵᏎ ᎤᏩᏂᎦᏢᎥ\n\n...,… ᎤᎵᏍᏆᎵᏎ ᎤᏩᏂᎦᏢ! ᎡᎳᏗ ᎤᏅᏥᎴ\nᏩᏯᏂᏓ!\nᏗᎦᎾᎥᎢᏣ ᏭᏓᏒᏍᏔᏁ...,0.49,0.81
4,Crane_And_Hummingbird_Race_sample1.png,ᏬᏂᏤᏓᎳ ᎤᏚᎵᏍᎨ\nᏗᎨᎦᏨᏍᏙᏗᎢ. ᏩᎴᎳ ᎥᏍᏊ\nᎤᏚᎵᏍᎨ ᏗᎨᎦᏨᏍᏙᏗᎢ...,"ᎠᎨᏳᏣᏃ ᎨᏒ, ᏩᎴᎳ ᎦᎸᏈᏗᏍᎨᎢ.\n\nᏝ ᏱᎦᎸᏈᏗᏍᎨ ᎧᏂᏤᏓᎳ.\n\n...",0.33,0.40
5,Crane_And_Hummingbird_Race_sample2.png,"“\nᏚᏁᎷ !\nᎸ5 79 ᏨᏟᏑᎴᎢ. ᏨᏍᎣᎾ ᏲᎪ ᎢᎢ.\nᎡ Ꭹ ᏓᎲ. 1,...",ᎧᏂᏤᏓᎳ ᎠᏁᏟᏗᏍᎨ ᎤᎸᏈᏙᏗ Ꮎ\nᎠᎨᏳᏣ.\nᎧᏂᏤᏓᎳ ᏚᏃᎩᏍᏓᏁᎴ ᎠᎨᏳ...,1.67,3.69
6,Diary.png,\n\n) !\n\n! 120 Ꮴ ᏟᏂᎶᏑᏫᏒᎾᏮ ᏙᏧᎮᏤᏩᏌᏙᏚ ” ’\n!...,ᎠᏂᏍᎬᏘ 31Ꮑ 1927 ᎯᎸᏍᎩ ᎢᎧᏁᏥ ᏓᎪᏪᎳᏂ ᎪᎯ ᎢᎪᎯ ᎠᏆᏓᏂᎳᏉ Ᏹ...,3.62,4.42
7,EncyclopediaBrown_sample.png,Ꮞ6\n\nᎬᏂᏟᎩᏟᎫᏣᎠᎾᏄᎥᏎᎸ ᎤᏬᏗᎨ\n\nᎤᎬᏫᏳᏌᏕᎩ ᎤᏬᏗᎨ ᎤᏗᏍᏛᏂ...,46 Encyclopedia ᎤᏬᏗᎨ\t \nᎤᎬᏫᏳᏌᏕᎩ\tᎤᏬᏗᎨ ᎤᏗᏍᏛᏂᎴ...,0.21,0.76
8,Halloween_sample1.png,\n \n\nᏚᏂᎾᏗ ᏦᏍᎪ ᏐᏊ ᎪᎢᎦ.\nᎠᏂᏥᏍᎩᎵ ᎠᏂᎾᏅᎪᎰᎢ\nᎪᎢ...,ᏍᎩᎵᏤ ᏙᎯ ᎡᏓᏍᏗ ᎢᎦ\n\nᏚᏂᎾᏗ ᏦᏍᎪ ᏐᏊ ᎪᎢᎦ.\nᎠᏂᏥᏍᎩᎵ ᎠᏂ...,0.20,0.50
9,Halloween_sample2.png,ᎠᏂᏥᏍᎩᎵ ᎠᏂᎾᏅᎪᎬ\nᏗᎧᏃᎮᏍᎩ\n\nᏍᏈᏍᏓ ᏕᏗᎪᏩᏘᏍᎪ ᏍᎩᎵ\nᎤᏂᏅ...,ᎠᏂᏥᏍᎩᎵ ᎠᏂᎾᏅᎪE\nᏗᎧᏃᎮᏍᎩ\nᏍᏈᏍᏓ ᏕᏗᎪᏩᏘᏍᎪ ᏍᎩᎵ\nᎤᏂᏅᎪᎢ...,0.10,0.25


In [72]:
df_output_screenshot = calculate_tesseract_ocr("./OCR/screenshot/")

Mean CER = 0.06679245283018868, Mean WER = 0.30320754716981135
Median of all setences CER = 0.49, Median of all setences WER = 0.865


In [73]:
df_output_screenshot

,img_filename,ocr_output,ref_text,cer,wer
0,Charlie_Brown_Christmas_sample1_shot1.png,"“Ꮩ ᎢᏳᏍᏗ ᏣᏚᎵᎭ’” ᎤᏛᏛᏅ ᎷᏏ.\n\n“ᏥᏳᎪᏗ ᎢᎦᏛᏁᏗ,” ᎤᏛᏅ Ꮳ...","“Ꮩ ᎢᏳᏍᏗ ᏣᏚᎵᎭ?” ᎤᏛᏛᏅ ᎷᏏ.\n\n“ᏥᏳᎪᏗ ᎢᎦᏛᏁᏗ,” ᎤᏛᏅ Ꮳ...",0.03,0.19
1,Charlie_Brown_Christmas_sample2_shot1.png,ᏎᎩᏊᏃ ᏂᎦᏓ ᏧᎾᏁᎶᏗ ᏗᎪᏪᎵ ᏚᏂᎾᎩᏒ. ᏃᎢ\nᎤᏍᏆᎸᎲ ᏣᎵ ᎤᎧᏗᎨ Ꮷ...,ᏎᎩᏊᏃ ᏂᎦᏓ ᏧᎾᏁᎶᏗ ᏗᎪᏪᎵ ᏚᏂᎾᎩᏒ. ᏃᎢ\nᎤᏍᏆᎸᎲ ᏣᎵ ᎤᏬᏗᎨ Ꮷ...,0.03,0.16
2,ClimbingTheAppleTree_sample1_shot1.png,"ᎠᏥᏃᎯᏎᎲ ᎢᏳᏩᏁᏗ ᎤᏣᎴᏍᏓ, ᎤᏩᎨᏫᏎᎢ\nᎤᏪᎯᏍᏓᏁᎲ ᎧᏃᎨᏂ.\n\n“...","ᎠᏥᏃᎯᏎᎲ ᎢᏳᏩᏁᏗ ᎤᏣᎴᏍᏓ, ᎤᏩᎨᏫᏎᎢ\nᎤᏪᎯᏍᏓᏁᎲ ᎧᏃᎨᏂ.\n\n“...",0.00,0.00
3,ClimbingTheAppleTree_sample1_shot2.png,“ᏃᏗ ᏓᏥᏇᏅᏂ\nᎤᏬᏍᎩᎵ ᎦᏢᏍᏗ\nᎯᏃᎨᏂ. ᎣᏍᏓ ᏛᏍᏕᎸ\nᎤᏣᎴᏍᏓ Ꭶ...,“ᏃᏗ ᏓᏥᏇᏅᏂ\nᎤᏬᏍᎩᎵ ᎦᏢᏍᏗ\nᎯᏃᎨᏂ. ᎣᏍᏓ ᏛᏍᏕᎸ\nᎤᏣᎴᏍᏓ Ꭶ...,0.00,0.00
4,ClimbingTheAppleTree_sample1_shot3.png,ᏄᏓᎴ ᎦᏢᏍᏙᏗ ᏭᎾᎩᏎ ᎦᏂᎦᏔ ᎡᎹᏂᏓ.\n,ᏄᏓᎴ ᎦᏢᏍᏙᏗ ᏭᎾᎩᏎ ᎦᏂᎦᏔ ᎡᎹᏂᏓ.,0.00,0.00
5,ClimbingTheAppleTree_sample1_shot4.png,"“Ꭰ3Ꮾ ᏓᎬᏩᏝᏂ\nᎦᏢᏍᏙᏗ ᎠᎬᏱ,\n\nᏃᏗ ᏓᏥᏇᏅᏂ ᎦᏚᎢ\nᎤᏬᏍᎩᎵ ...","“ᎠᎼ ᏓᎬᏩᏝᏂ\nᎦᏢᏍᏙᏗ ᎠᎬᏱ,\n\nᏃᏗ ᏓᏥᏇᏅᏂ ᎦᏚᎢ\nᎤᏬᏍᎩᎵ Ꮳ...",0.03,0.11
6,ClimbingTheAppleTree_sample2_shot1.png,..ᎤᎵᏍᏆᎵᏎ ᎤᏩᏂᎦᏢ:Ꭵ ᎡᎳᏗ ᎤᏅᏥᎴ\nᏩᏯᏂᏓᎥ\n\nᏗᎦᎾᎥᎢᏣ ᏭᏓᏒ...,… ᎤᎵᏍᏆᎵᏎ ᎤᏩᏂᎦᏢ! ᎡᎳᏗ ᎤᏅᏥᎴ\nᏩᏯᏂᏓ!\nᏗᎦᎾᎥᎢᏣ ᏭᏓᏒᏍᏔᏁ...,0.10,0.62
7,ClimbingTheAppleTree_sample2_shot2.png,"ᎨᏍᏗ ᎩᎳᏈᏴ ᎪᎱᏍᏗ ᏳᏛᎾ ᏩᏯᏂᏓ.\nᎩᎳᏗ ᎤᎵᏰᏔᏁᎢ, “ᏥᏃᎨᏂᎥ ᏥᏃ...","ᎨᏍᏗ ᎩᎳᏈᏴ ᎪᎱᏍᏗ ᏳᏛᎾ ᏩᏯᏂᏓ.\nᎩᎳᏗ ᎤᎵᏰᏔᏁᎢ, “ᏥᏃᎨᏂ! ᏥᏃ...",0.06,0.25
8,ClimbingTheAppleTree_sample2_shot3.png,"ᏎᎦ ᎾᎥᏂ ᎤᎵᏍᏛᏧᏁᎢ ᎦᎦ, ᏃᏗ,\n“ᏣᏙᏓ ᏓᏥᏯᏅᎮᏏ,” ᎤᏛᏁ…….\n","ᏎᎦ ᎾᎥᏂ ᎤᎵᏍᏛᎴᏁᎢ ᏚᎦ, ᏃᏗ,\n“ᏣᏙᏓ ᏓᏥᏯᏅᎮᏏ,” ᎤᏛᏁ.\n\n",0.10,0.43
9,Crane_And_Hummingbird_Race_sample1_shot1.png,"ᎠᎨᏳᏣᏃ ᎨᏒ, ᏩᎴᎳ ᎦᎸᏈᏗᏍᎨᎢ.\n\nᏝ ᏱᎦᎸᏈᏗᏍᎨ ᎧᏂᏤᏓᎳ.\n","ᎠᎨᏳᏣᏃ ᎨᏒ, ᏩᎴᎳ ᎦᎸᏈᏗᏍᎨᎢ.\n\nᏝ ᏱᎦᎸᏈᏗᏍᎨ ᎧᏂᏤᏓᎳ.\n",0.00,0.00


In [ ]:
!pip install --upgrade google-cloud-vision

In [ ]:
def detect_text(path):
    """Detects text in the file."""
    from google.cloud import vision
    import io
    client = vision.ImageAnnotatorClient()

    with io.open(path, 'rb') as image_file:
        content = image_file.read()

    image = vision.Image(content=content)

    response = client.text_detection(image=image)
    texts = response.text_annotations
    print('Texts:')

    for text in texts:
        print('\n"{}"'.format(text.description))

        vertices = (['({},{})'.format(vertex.x, vertex.y)
                    for vertex in text.bounding_poly.vertices])

        print('bounds: {}'.format(','.join(vertices)))

    if response.error.message:
        raise Exception(
            '{}\nFor more info on error messages, check: '
            'https://cloud.google.com/apis/design/errors'.format(
                response.error.message))

In [ ]:
detect_text("OCR/original/Charlie_Brown_Christmas_sample1.png")

In [40]:
!git stash
!git stash drop

Saved working directory and index state WIP on master: ff3fd7f first commit
Dropped refs/stash@{0} (0b0dd53353c2e3b3ee04f6468c7382ba0f5b932b)


In [41]:
!git status

On branch master
Your branch is up to date with 'origin/master'.

nothing to commit, working tree clean


**Tokenization**

1. Character as a Token

In [61]:
def translate_cherokee_to_latin(text):
    # Create a dictionary of Cherokee characters and their corresponding Latin characters
    cherokee_to_latin = {
        "Ꭰ": "a", "Ꭱ": "e", "Ꭲ": "i", "Ꭳ": "o", "Ꭴ": "u", "Ꭵ": "v", "Ꭶ": "ga", "Ꭷ": "ka", "Ꭸ": "ge",
        "Ꭹ": "gi", "Ꭺ": "go", "Ꭻ": "gu", "Ꭼ": "gv", "Ꭽ": "ha", "Ꭾ": "he", "Ꭿ": "hi", "Ꮀ": "ho", "Ꮁ": "hu",
        "Ꮂ": "hv", "Ꮃ": "la", "Ꮄ": "le", "Ꮅ": "li", "Ꮆ": "lo", "Ꮇ": "lu", "Ꮈ": "lv", "Ꮉ": "ma", "Ꮊ": "me",
        "Ꮋ": "mi", "Ꮌ": "mo", "Ꮍ": "mu", "Ꮎ": "na", "Ꮏ": "hna", "Ꮐ": "na", "Ꮑ": "ne", "Ꮒ": "ni", "Ꮓ": "no",
        "Ꮔ": "nu", "Ꮕ": "nv", "Ꮖ": "qua", "Ꮗ": "que", "Ꮘ": "qui", "Ꮙ": "quo", "Ꮚ": "quu", "Ꮛ": "quv", "Ꮜ": "sa",
        "Ꮝ": "s", "Ꮞ": "se", "Ꮟ": "si", "Ꮠ": "so", "Ꮡ": "su", "Ꮢ": "sv", "Ꮣ": "da", "Ꮤ": "ta", "Ꮥ": "de",
        "Ꮦ": "te", "Ꮧ": "di", "Ꮨ": "ti", "Ꮩ": "do", "Ꮪ": "du", "Ꮫ": "dv", "Ꮬ": "dla", "Ꮭ": "tla", "Ꮮ": "tle",
        "Ꮯ": "tli", "Ꮰ": "tlo", "Ꮱ": "tlu", "Ꮲ": "tlv", "Ꮳ": "tsa", "Ꮴ": "tse", "Ꮵ": "tsi", "Ꮶ": "tso",
        "Ꮷ": "tsu", "Ꮸ": "tsv", "Ꮹ": "wa", "Ꮺ": "we", "Ꮻ": "wi", "Ꮼ": "wo", "Ꮽ": "wu", "Ꮾ": "wv", "Ꮿ": "ya",
        "Ᏸ": "ye", "Ᏹ": "yi", "Ᏺ": "yo", "Ᏻ": "yu", "Ᏼ": "yv"," ":" "
    }
    return cherokee_to_latin[text]


In [62]:
text = "ᎠᏂᎩᏚᏩᏙᎯ ᎠᏂᎩᏚᏩᏙᎯ ᎠᏂᎩᏚᏩᏙᎯ"
translated_text=""
for t in text:
    translated_text += translate_cherokee_to_latin(t)+"-"
print(translated_text)

a-ni-gi-du-wa-do-hi- -a-ni-gi-du-wa-do-hi- -a-ni-gi-du-wa-do-hi-


In [54]:
import nltk
nltk.download("punkt")

text = "ᎠᏂᎩᏚᏩᏙᎯ ᎠᏂᎩᏚᏩᏙᎯ ᎠᏂᎩᏚᏩᏙᎯ"

# Tokenize the text using NLTK's word_tokenize function
tokens = nltk.word_tokenize(text)
print(tokens)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


['ᎠᏂᎩᏚᏩᏙᎯ', 'ᎠᏂᎩᏚᏩᏙᎯ', 'ᎠᏂᎩᏚᏩᏙᎯ']


In [ ]:

# Download the Cherokee corpus if it's not already downloaded
nltk.download('cherokee')

# Access the Cherokee corpus
corpus = nltk.corpus.cherokee.words()

# Print the first 10 words of the corpus
print(corpus[:10])
